# Import 

In [1]:
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Prepare data

In [2]:
train_dir = "../input/emotion-detection-fer/train"
test_dir = "../input/emotion-detection-fer/test"

In [3]:
import tensorflow as tf

train_datagen = ImageDataGenerator(#rotation_range = 180,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   horizontal_flip = True,
                                   rescale = 1./255,
                                   #zoom_range = 0.2,
                                   validation_split = 0.2)
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                        validation_split = 0.2)

In [4]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (48, 48),
                                                    batch_size = 64,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training")

validation_generator = validation_datagen.flow_from_directory(directory = test_dir,
                                                              target_size = (48, 48),
                                                              batch_size = 64,
                                                              color_mode = "grayscale",
                                                              class_mode = "categorical",
                                                              subset = "validation")

In [5]:
plt.imshow(train_generator.next()[0][0], cmap="gray")

In [6]:
for x in train_generator.next():
    print(x[0])
    

# Model

In [15]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Input((48, 48, 1)))
model.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(7, activation="softmax"))

In [16]:
model.summary()

In [17]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])#metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.Recall(thresholds=0)])

In [26]:
from tensorflow.keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor="val_loss",
                         mode="min",
                         patience=2)

In [27]:
history = model.fit(x=train_generator,
                    epochs=100,
                    validation_data=validation_generator,
                    callbacks=[callback])

In [28]:
model.save("img_to_emotion.h5")